# بهینه‌سازی مدل MLP برای تشخیص بیماری قلبی
در این نوت‌بوک، مدل MLP را با استفاده از GridSearchCV بهینه می‌کنیم.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
import matplotlib.pyplot as plt
import numpy as np


In [8]:
# بارگذاری و پیش‌پردازش داده‌ها
df = pd.read_csv('Heart_disease_cleveland_new.csv')
X = df.drop('target', axis=1)
y = df['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = X_train
y_train = y_train.values


In [9]:
# تعریف مدل MLP برای GridSearchCV
def build_model(hidden_layers=1, neurons=32, learning_rate=0.01, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# تبدیل به KerasClassifier
model = KerasClassifier(
    model=build_model,
    hidden_layers=1,
    neurons=32,
    learning_rate=0.01,
    activation='relu',
    epochs=20,
    batch_size=32,
    verbose=0
)

In [10]:
# تنظیم فضای جستجو برای GridSearchCV
param_grid = {
    'hidden_layers': [1, 2],
    'neurons': [16, 32, 64],
    'learning_rate': [0.001, 0.005, 0.01],
    'activation': ['relu', 'tanh'],
    'epochs': [20, 30],
    'batch_size': [16, 32]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

In [11]:
# اجرای GridSearchCV
grid_result = grid.fit(X_train, y_train)


print(f'Best Accuracy: {grid_result.best_score_}')
print(f'Best Parameters: {grid_result.best_params_}')

Fitting 3 folds for each of 144 candidates, totalling 432 fits


c:\Users\Emad\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\Emad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Accuracy: 0.8512345679012346
Best Parameters: {'activation': 'relu', 'batch_size': 16, 'epochs': 30, 'hidden_layers': 1, 'learning_rate': 0.005, 'neurons': 64}


In [12]:
# نمایش نتایج جستجو
results_df = pd.DataFrame(grid_result.cv_results_)
results_df = results_df.sort_values(by='mean_test_score', ascending=False)
results_df[['params', 'mean_test_score', 'std_test_score']].head(10)

,params,mean_test_score,std_test_score
23,"{'activation': 'relu', 'batch_size': 16, 'epoc...",0.851235,0.010118
141,"{'activation': 'tanh', 'batch_size': 32, 'epoc...",0.847068,0.021265
125,"{'activation': 'tanh', 'batch_size': 32, 'epoc...",0.838889,0.009334
34,"{'activation': 'relu', 'batch_size': 16, 'epoc...",0.838786,0.010974
98,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.826440,0.010132
142,"{'activation': 'tanh', 'batch_size': 32, 'epoc...",0.822428,0.020137
106,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.822274,0.006911
88,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.822222,0.016502
77,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.822119,0.036589
80,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.818210,0.020724
